Dette programmet gjør en stilistisk analyse av strategiene med embeddings (topografisk analyse, se Toubia et al., 2021). Dette er for å sammenlikne og se om noen skiller seg sterkt fra de andre, men er ikke ellers veldig interessant for våre formål. Det er likevel en betydelig (om noe uinteressant) utvidelse av nøkkelordsanalysen.

Også dette programmet tar evigheter å kjøre (først og fremst tar embeddingen grusomt lang tid (uten GPU/cuda), og deretter løser vi noen halvtunge optimeringsproblemer (TSP og volumberegninger, se Topography.py)).

In [1]:
from Topography import Topography

import pandas as pd
import os
import gc

load c:\Users\jonas\anaconda3\envs\AIbase\Lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\jonas\anaconda3\envs\AIbase\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\jonas\anaconda3\envs\AIbase\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\jonas\anaconda3\envs\AIbase\Lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\jonas\anaconda3\envs\AIbase\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\jonas\anaconda3\envs\AIbase\Lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\jonas\anaconda3\envs\AIbase\Lib\site-packages\ortools\.libs\ortools.dll...


c:\Users\jonas\anaconda3\envs\AIbase\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
folder = "./txt" # change source folder here
chunksize = 500 # change chunksize here


data = []

for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)

    if os.path.isfile(file_path):
        # Analysis
        text = Topography(file_path)
        text.embed(chunksize=chunksize)
        embeddings = text.embeddings
        volume = text.volume()
        speed, circuitousness = text.speed()
        data.append([filename, embeddings, volume, speed, circuitousness])

        del text
        gc.collect() # python's automatic gc is bad

    else:
        msg = "-"*50 + f"\nFound no file with {filename}\n" + "-"*50 + "\n"
        print(msg)
    
df = pd.DataFrame(data, columns=['Country', f'Embeddings ({chunksize})', 'volume', 'speed', 'circuitousness'])
print(df)


              Country                                   Embeddings (500)  \
0  Brazil_summary.txt  [[-0.21755669, 0.33351547, -0.7115478, -0.2464...   
1           China.txt  [[-0.09726964, 0.08906725, -0.09776214, -0.488...   
2              EU.txt  [[0.030849582, 0.33885244, -1.0729142, 0.29462...   
3          Norway.txt  [[0.6098222, 0.508988, -0.12597772, 0.23846106...   
4          Poland.txt  [[-0.15160954, -0.050911434, 0.23864263, 0.168...   
5             USA.txt  [[0.2750075, -0.16566397, 0.05493906, 0.205856...   

     volume     speed  circuitousness  
0  1.865337  2.299453        0.060147  
1  2.003385  2.433337        0.062631  
2  1.922355  2.311660        0.052566  
3  2.027906  2.439491        0.058022  
4  1.906332  2.360219        0.104344  
5  1.938302  2.306274        0.030646  


In [4]:
df.to_csv("embeddings.csv", index=False)